In [31]:
# импорт всех необходимых библиотек
import os
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.exceptions import ConvergenceWarning

# Игнорировать предупреждения о сходимости
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import pandas as pd

parent_directory = os.path.dirname(os.getcwd())  # Получаем родительский каталог
os.chdir(parent_directory)  # Переходим в родительский каталог
print(os.getcwd())  # Проверяем, что мы находимся в родительском каталоге

/Users/elizavetachefanova


In [2]:
df = pd.read_csv(f'{parent_directory}/data/Applicant-details.csv', sep=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   Applicant_ID                 100000 non-null  int64 
 1   Annual_Income                100000 non-null  int64 
 2   Applicant_Age                100000 non-null  int64 
 3   Work_Experience              100000 non-null  int64 
 4   Marital_Status               100000 non-null  object
 5   House_Ownership              100000 non-null  object
 6   Vehicle_Ownership(car)       100000 non-null  object
 7   Occupation                   100000 non-null  object
 8   Residence_City               100000 non-null  object
 9   Residence_State              100000 non-null  object
 10  Years_in_Current_Employment  100000 non-null  int64 
 11  Years_in_Current_Residence   100000 non-null  int64 
 12  Loan_Default_Risk            100000 non-null  int64 
dtypes: int64(7), ob

In [47]:
# Список категориальных столбцов, которые нужно закодировать
categorical_columns = ['Residence_State', 'Residence_City', 'Occupation', 'Vehicle_Ownership(car)', 'House_Ownership', 'Marital_Status']  # и так далее

# Применяем One-Hot Encoding ко всем указанным столбцам
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Теперь df_encoded содержит новые столбцы для каждой категории в указанных столбцах,
# а исходные категориальные столбцы удалены.

X = df_encoded.drop(['Applicant_ID', 'Loan_Default_Risk'], axis=1) # Убираем столбец с целевой переменной из данных
y = df_encoded['Loan_Default_Risk'] # Целевая переменная

# # Разбиение данных на обучающую и тестовую выборки с сохранением пропорций классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [48]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import time
import sys
import pickle

# Засекаем время начала обучения
start_time = time.time()

# Создание модели логистической регрессии
model = LogisticRegression(max_iter=1000)  # Увеличиваем количество итераций

# Обучение модели на обучающей выборке
model.fit(X_train, y_train)

# Время, затраченное на обучение
training_time = time.time() - start_time

# Засекаем время начала предсказания
start_time = time.time()

# Предсказание на тестовой выборке
predictions = model.predict(X_test)
predictions_proba = model.predict_proba(X_test)[:, 1]

# Время, затраченное на предсказание
prediction_time = time.time() - start_time

# Оценка производительности модели
accuracy = accuracy_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, predictions_proba)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Оценка объема памяти, занимаемой моделью
model_pickle = pickle.dumps(model)
model_size = sys.getsizeof(model_pickle)

print(f"Точность модели: {accuracy}")
print(f"ROC AUC: {roc_auc}")
print(f"Точность (Precision): {precision}")
print(f"Полнота (Recall): {recall}")
print(f"F1 score: {f1}")
print(f"Время обучения: {training_time} секунд")
print(f"Время предсказания: {prediction_time} секунд")
print(f"Объем памяти: {model_size} байт")


Точность модели: 0.87005
ROC AUC: 0.5207780069690793
Точность (Precision): 0.0
Полнота (Recall): 0.0
F1 score: 0.0
Время обучения: 12.223979949951172 секунд
Время предсказания: 0.31037163734436035 секунд
Объем памяти: 15034 байт


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
predictions.sum()

0

In [39]:
model.predict([X_test.iloc[0]])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0])